# 数据源

## 相关技术

## 数据预处理

# 图谱构建

## 知识抽取

In [2]:
# -*- coding: utf-8 -*-
import re
file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.txt'
out_file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.csv'
dict_file_name = "D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点字典.txt"
pattern = r'[:|：]' 

# 得到当前层级数
def get_level(line):
    ret_level = -1
    lev = {
        1 : r'第[一|二|三|四|五|六]章',
        2 : r'[一|二|三|四|五|六|七|八|九]、',
        3 : r'（[一|二|三|四|五|六]）',
        4 : r'\d+、',
        5 : r'[⑴|⑵|⑶|⑷|⑸|⑹|⑺|⑻]'}
    for key in lev.keys():
        if re.match(lev[key],line):
            break
            
    if key:
        ret_level = key
    return ret_level, lev
details = []

with open(file_name,'r',encoding='utf-8') as f:
    current_lev_map = {}
    for line in f.readlines():
        line = line.strip('\n').strip()
        if len(line) > 0:
            result = re.split(pattern, line, maxsplit=1)
            level,lev_define = get_level(line)
            result = [x.strip('\n') for x in result]
#             print('level --> ',level, ' result --> ', result[0])
            result[0] = re.split(lev_define[level],result[0],maxsplit=1)[1]
            current_lev_map[level] = result[0]
            up_title = ''
            if level >= 2:
                up_title = current_lev_map[level-1]
            details.append([up_title, result[0].strip(), result[1].strip() if len(result) > 1 else '' ])
#             print('title:{}-->{}'.format(up_title, result))
import pandas as pd
import numpy as np
dataframe = pd.DataFrame(np.array(details),columns=['title','name','detail'])
dataframe.to_csv(out_file_name,index=False,sep=',',encoding='utf_8_sig')

with open(dict_file_name, 'w', encoding='utf-8') as f:
    content = dataframe.name.values.tolist()
    content.append('圆点')
    text = '\n'.join(content)
    f.writelines(text)
        
print('over !')

'''
注意生成的csv文件需保存到 neo4j 应用目录import 下
执行脚本：
LOAD CSV WITH HEADERS FROM "file:/math.csv" AS row
CREATE (n:Knowledge)
set n=row;
'''
    

over !


'\n注意生成的csv文件需保存到 neo4j 应用目录import 下\n执行脚本：\nLOAD CSV WITH HEADERS FROM "file:/math.csv" AS row\nCREATE (n:Knowledge)\nset n=row;\n'

## 关系抽取 

### 简单关系抽取

In [3]:

'''
1、根据文档结构建立PARTOF关系
执行脚本：
MATCH (a:Knowledge), (b:Knowledge) 
   WHERE a.name = b.title
CREATE (a)-[:PARTOF]->(b) 
RETURN a,b 

2、根据CSV知识点内容中检索方式，建立USE关系
处理流程（极简流程）：
1） 对内容分词，然后通过分词去匹配， 注意（这里关系有可能是引用和使用两种，怎么样进行区分？？？）  分词工具包： https://github.com/lancopku/pkuseg-python
'''
import pandas as pd
import pkuseg
import numpy as np
dict_file_name = "D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点字典.txt"
file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.csv'
wordseg = pkuseg.pkuseg(user_dict=dict_file_name)
dataframe = pd.read_csv(file_name)

kn_dict = None
with open(dict_file_name,'r',encoding='utf-8') as f:
    lines = f.readlines()
    kn_dict = [x.strip() for x in lines]

# print(kn_dict)
# 检测前置情况
def is_pre(cut_text):
    pre_kn = []
    cut_text = np.array(cut_text)
    for item in kn_dict:
        result = np.where(cut_text==item)
        if len(result[0]) > 0:
            pre_kn.append(item)
    return pre_kn

for idx, row in dataframe.iterrows():
    detail = row['detail']
    if isinstance(detail,str) :
        text = wordseg.cut(detail)
        print('kn -> {} pre kn : {} cut txt : {}'.format(row['name'], is_pre(text), text))




kn -> 整数的意义 pre kn : ['整数', '自然数'] cut txt : ['自然数', '和', '0', '都', '是', '整数', '。']
kn -> 自然数 pre kn : ['自然数'] cut txt : ['我们', '在', '数', '物体', '的', '时候', '，', '用来', '表示', '物体', '个数', '的', '1，2，3……', '叫做', '自然数', '。', '一个', '物体', '也', '没有', '，', '用', '0', '表示', '。', '0', '也', '是', '自然数', '。']
kn -> 计数单位 pre kn : ['计数单位'] cut txt : ['一', '（', '个', '）', '、', '十', '、', '百', '、', '千', '、', '万', '、', '十万', '、', '百万', '、', '千万', '、', '亿', '……', '都', '是', '计数单位', '。', '其中', '“', '一', '”', '是', '计数', '的', '基本', '单位', '。', '10', '个', '1', '是', '10', '，', '10', '个', '10', '是', '100', '……', '每', '相邻', '两', '个', '计数单位', '之间', '的', '进率', '都', '是', '10', '。', '这样', '的', '计数法', '叫做', '十进制', '计数法', '。']
kn -> 数位 pre kn : ['计数单位', '数位'] cut txt : ['计数单位', '按照', '一定', '的', '顺序', '排列', '起来', '，', '它们', '所', '占', '的', '位置', '叫做', '数位', '。']
kn -> 整数的读法 pre kn : ['数位'] cut txt : ['从', '高位', '到', '低位', '，', '一级', '一级', '地', '读', '。', '读', '亿', '级', '、', '万', '级', '时', '，', '先', '按照', '个', '级', '的', '读法', '去

kn -> 真分数 pre kn : ['分数', '真分数'] cut txt : ['分子', '比分', '母', '小', '的', '分数', '叫做', '真分数', '。', '真分数', '小于', '1', '。']
kn -> 假分数 pre kn : ['分数', '假分数'] cut txt : ['分子', '比分', '母大', '或者', '分子', '和', '分母', '相等', '的', '分数', '，', '叫做', '假分数', '。', '假分数', '大于', '或', '等于', '1', '。']
kn -> 带分数 pre kn : ['整数', '真分数', '假分数', '带分数'] cut txt : ['假分数', '可以', '写成', '整数', '与', '真分数', '合成', '的', '数', '，', '通常', '叫做', '带分数', '。']
kn -> 分数和除法的关系及分数的基本性质 pre kn : ['分数', '约分和通分', '分数的基本性质'] cut txt : ['除法', '是', '一', '种', '运算', '，', '有', '运算', '符号', '；', '分数', '是', '一', '种', '数', '。', '因此', '，', '一般', '应', '叙述', '为', '被', '除数', '相当于', '分子', '，', '而', '不', '能', '说成', '被', '除数', '就是', '分子', '。', '由于', '分数和除法', '有', '密切', '的', '关系', '，', '根据', '除法', '中', '“', '商', '不', '变', '”', '的', '性质', '可', '得出', '分数的基本性质', '。', '分数的分', '子', '和', '分母', '都', '乘以', '或者', '除以', '相同', '的', '数', '（', '0', '除外', '）', '，', '分数的', '大小', '不', '变', '，', '这', '叫做', '分数的基本性质', '，', '它', '是', '约分和通分', '的', '依据', '。']
kn -> 最简分数 pre k

kn -> 整除的特征 pre kn : [] cut txt : ['个', '位', '上', '是', '0', '、', '2', '、', '4', '、', '6', '、', '8', '的', '数', '，', '都', '能', '被', '2', '整除', '。', '个', '位', '上', '是', '0', '或', '5', '的', '数', '，', '都', '能', '被', '5', '整除', '。', '一个', '数', '的', '各位', '上', '的', '数', '的', '和', '能', '被', '3', '整除', '，', '这个', '数', '就', '能', '被', '3', '整除', '。', '一个', '数', '各位', '数', '上', '的', '和', '能', '被', '9', '整除', '，', '这个', '数', '就', '能', '被', '9', '整除', '。', '能', '被', '3', '整除', '的', '数', '不', '一定', '能', '被', '9', '整除', '，', '但是', '能', '被', '9', '整除', '的', '数', '一定', '能', '被', '3', '整除', '。', '一个', '数', '的', '末', '两', '位数', '能', '被', '4', '（', '或', '25', '）', '整除', '，', '这个', '数', '就', '能', '被', '4', '（', '或', '25', '）', '整除', '。', '一个', '数', '的', '末', '三', '位数', '能', '被', '8', '（', '或', '125', '）', '整除', '，', '这个', '数', '就', '能', '被', '8', '（', '或', '125', '）', '整除', '。']
kn -> 质数 pre kn : ['质数'] cut txt : ['一个', '数', '，', '如果', '只有', '1', '和', '它', '本身', '两', '个', '约数', '，', '这样', '的', '数', '叫做', '质

kn -> 分数乘法 pre kn : ['整数乘法'] cut txt : ['分数乘法的', '意义', '与', '整数乘法', '的', '意义', '相同', '，', '就是', '求', '几', '个', '相同', '加数', '和', '的', '简便', '运算', '。']
kn -> 分数除法 pre kn : ['整数除法'] cut txt : ['分数除法的', '意义', '与', '整数除法', '的', '意义', '相同', '。', '就是', '已', '知', '两', '个', '因数', '的', '积与', '其中', '一个', '因数', '，', '求', '另', '一个', '因数', '的', '运算', '。']
kn -> 加法交换律 pre kn : [] cut txt : ['两', '个数', '相加', '，', '交换', '加数', '的', '位置', '，', '它们', '的', '和', '不', '变', '，', '即', 'a+b=b+a', '。']
kn -> 加法结合律 pre kn : [] cut txt : ['三', '个数', '相加', '，', '先', '把', '前', '两', '个数', '相加', '，', '再', '加上', '第三', '个数', '；', '或者', '先', '把', '后', '两', '个数', '相加', '，', '再', '和', '第一', '个数', '相加', '它们', '的', '和', '不', '变', '，', '即', '（', 'a+b)+c=a+', '(', 'b+c', ')', '。']
kn -> 乘法交换律 pre kn : [] cut txt : ['两', '个数', '相乘', '，', '交换', '因数', '的', '位置', '它们', '的', '积', '不', '变', '，', '即', 'a×b=b×a', '。']
kn -> 乘法结合律 pre kn : [] cut txt : ['三', '个数', '相乘', '，', '先', '把', '前', '两', '个数', '相乘', '，', '再', '乘以', '第三', '个数', '

kn -> 平均数问题 pre kn : ['小数', '时间'] cut txt : ['平均数', '是', '等', '分除法', '的', '发展', '。', '解题', '关键', '：', '在于', '确定', '总数量', '和', '与', '之', '相', '对应', '的', '总', '份数', '。', '算术', '平均数', '：', '已', '知', '几', '个', '不', '相等', '的', '同类量', '和', '与', '之', '相', '对应', '的', '份数', '，', '求', '平均', '每份', '是', '多少', '。', '数量', '关系式', '：', '数量', '之', '和', '÷', '数量', '的', '个数', '=', '算术', '平均数', '。', '加权', '平均数', '：', '已', '知', '两', '个', '以上', '若干', '份', '的', '平均数', '，', '求', '总', '平均数', '是', '多少', '。', '数量', '关系式', '（', '部分', '平均', '数×权', '数', '）', '的', '总和÷', '（', '权数', '的', '和', '）', '=', '加权', '平均数', '。', '差额', '平均数', '：', '是', '把', '各个', '大于', '或', '小于', '标准数', '的', '部分', '之', '和', '被', '总份', '数', '均分', '，', '求', '的', '是', '标准数', '与', '各', '数', '相差', '之', '和', '的', '平均数', '。', '数量', '关系式', '：', '（', '大数', '－', '小数', '）', '÷', '2', '=', '小数', '应', '得数', '最', '大数', '与', '各', '数', '之', '差', '的', '和', '÷', '总份数', '=', '最', '大', '数', '应', '给', '数', '最', '大数', '与', '个数', '之', '差', '的', '和', '÷', '总份数', '=',

...

kn -> 年龄问题 pre kn : ['差倍问题', '年龄问题', '时间'] cut txt : ['将', '差', '为', '一定', '值', '的', '两', '个数', '作为', '题', '中', '的', '一个', '条件', '，', '这种', '应用', '题', '被', '称为', '“', '年龄问题', '”', '。', '解题', '关键', '：', '年龄问题', '与', '和', '差', '、', '和', '倍', '、', '差倍问题', '类似', '，', '主要', '特点', '是', '随着', '时间', '的', '变化', '，', '年岁', '不断', '增长', '，', '但', '大小', '两', '个', '不同', '年龄', '的', '差', '是', '不', '会', '改变', '的', '，', '因此', '，', '年龄问题', '是', '一', '种', '“', '差不变', '”', '的', '问题', '，', '解题', '时', '，', '要', '善于', '利用', '差不变', '的', '特点', '。', '例', '父亲', '48', '岁', '，', '儿子', '21', '岁', '。', '问', '几', '年前', '父亲', '的', '年龄', '是', '儿子', '的', '4', '倍', '？', '分析', '：', '父子', '的', '年龄', '差', '为', '48-21', '=', '27', '（', '岁', '）', '。', '由于', '几', '年前', '父亲', '年龄', '是', '儿子', '的', '4', '倍', '，', '可知', '父子', '年龄', '的', '倍数', '差', '是', '（', '4-1', '）', '倍', '。', '这样', '可以', '算出', '几', '年前', '父子', '的', '年龄', '，', '从而', '可以', '求出', '几', '年前', '父亲', '的', '年龄', '是', '儿子', '的', '4', '倍', '。', '列式', '为', '：', '21', '（',

kn -> 质量常用单位 pre kn : [] cut txt : ['*', '吨', 't', '*', '千克', 'kg', '*', '克', 'g']
kn -> 质量常用换算 pre kn : [] cut txt : ['*', '一', '吨', '=', '1000千', '克', '*', '1千', '克', '=', '1000', '克']
kn -> 什么是时间 pre kn : ['时间'] cut txt : ['是', '指', '有', '起点', '和', '终点', '的', '一', '段', '时间']
kn -> 时间常用单位 pre kn : [] cut txt : ['世纪', '、', '年', '、', '月', '、', '日', '、', '时', '、', '分', '、', '秒']
kn -> 时间单位换算 pre kn : [] cut txt : ['*', '1', '世纪', '=', '100年', '（', '公元', '1', '年', '—', '100年', '是', '第一', '世纪', '，', '公元', '1901', '—', '2000', '是', '第二十', '世纪', '）', '*', '平年一年', '365', '天', '，', '闰年', '一', '年', '366', '天', '。', '*', '1', '年', '12', '个', '月', '（', '一', '、', '三', '、', '五', '、', '七', '、', '八', '、', '十', '、', '十二', '是', '大月', '，', '大月', '有', '31', '天', '；', '四', '、', '六', '、', '九', '、', '十一', '是', '小月', '小月', '，', '小月', '有', '30', '天', '；', '平年', '2月', '有', '28', '天', '闰年', '2月', '有', '29', '天', '）', '*', '闰年', '年份', '是', '4', '的', '倍数', '，', '整', '百年', '份', '须', '是', '400', '的', '倍数', '。', '*

kn -> 比的性质 pre kn : [] cut txt : ['比', '的', '前项', '和', '后项', '同时', '乘上', '或者', '除以', '相同', '的', '数', '（', '0', '除外', '）', '，', '比值', '不', '变', '，', '这', '叫做', '比', '的', '基本', '性质', '。']
kn -> 求比值和化简比 pre kn : ['整数', '小数', '分数'] cut txt : ['求比值', '的', '方法', '：', '用', '比', '的', '前', '项除', '以后', '项', '，', '它', '的', '结果', '是', '一个', '数值', '可以', '是', '整数', '，', '也', '可以', '是', '小数', '或', '分数', '。', '根据', '比', '的', '基本', '性质', '可以', '把', '比', '化成', '最', '简单', '的', '整数', '比', '。', '它', '的', '结果', '必须', '是', '一个', '最', '简比', '，', '即', '前', '、', '后', '项', '是', '互质', '的', '数', '。']
kn -> 比例尺 pre kn : ['比例尺', '线段'] cut txt : ['图', '上', '距离', '：', '实际', '距离', '=', '比例尺', '要求', '会', '求', '比例尺', '；', '已', '知', '图', '上', '距离', '和', '比例尺', '求', '实际', '距离', '；', '已', '知', '实际', '距离', '和', '比例尺', '求图', '上', '距离', '。', '线段', '比例尺', '：', '在', '图', '上', '附有', '一', '条', '注有', '数目', '的', '线段', '，', '用来', '表示', '和', '地面', '上', '相', '对应', '的', '实际', '距离', '。']
kn -> 按比例分配 pre kn : ['按比例分配'] cut txt : ['在', '农业

kn -> 圆的特征 pre kn : ['长度', '线', '线段', '圆'] cut txt : ['圆', '是', '平面', '上', '的', '一', '种', '曲', '线', '图形', '。', '圆', '中心', '的', '一点', '叫做', '圆', '心', '。', '一般', '用', '字母o', '表示', '。', '半径', '：', '连接', '圆', '心', '和', '圆', '上', '任意', '一点', '的', '线段', '叫做', '半径', '。', '一般', '用r', '表示', '。', '在', '同一个', '圆', '里', '，', '有', '无数', '条', '半径', '，', '每', '条', '半径', '的', '长度', '都', '相等', '。', '通过', '圆', '心', '并且', '两端', '都', '在', '圆', '上', '的', '线段', '叫做', '直径', '。', '一般', '用d', '表示', '。', '同一个', '圆', '里', '有', '无数', '条', '直径', '，', '所有', '的', '直径', '都', '相等', '。', '同', '圆', '或', '等', '圆的', '直径', '都', '相等', '同一个', '圆', '里', '，', '直径', '等于', '两', '个', '半径', '的', '长度', '，', '即', 'd=2r', '。', '圆的', '大小', '由', '半径', '决定', '。', '圆', '有', '无数', '条', '对', '称轴', '。', '圆', '心', '确定', '圆的', '位置', '，', '半径', '确定', '圆的', '大小', '。']
kn -> 圆的画法 pre kn : ['圆'] cut txt : ['把', '圆', '规', '的', '两', '脚', '分开', '，', '定好', '两', '脚', '间', '的', '距离', '（', '即', '半径', '）', '；', '把', '有', '针尖', '的', '一', '只', '脚', '固定', '

kn -> 折线统计图 pre kn : ['长度', '时间', '线', '直线', '射线', '线段', '折线统计图'] cut txt : ['用', '一个', '单位', '长度', '表示', '一定', '的', '数量', '，', '根据', '数量', '的', '多少', '描出', '各', '点', '，', '然后', '把', '各', '点', '用', '线段', '顺次', '连接', '起来', '。', '优点', '：', '不但', '可以', '表示', '数量', '的', '多少', '，', '而且', '能够', '清楚', '地', '表示', '出', '数量', '增减', '变化', '的', '情况', '。', '注意', '：', '折线统计图', '的', '横轴', '表示', '不同', '的', '年份', '、', '月份', '等', '时间', '时', '，', '不同', '时间', '之间', '的', '距离', '要', '根据', '年份', '或', '月份', '的', '间隔', '来', '确定', '。', '制作', '折线统计图', '的', '一般', '步骤', ':', '（', '1', '）', '根据', '图纸', '的', '大小', '，', '画', '出', '两', '条', '互相', '垂直', '的', '射线', '。', '（', '2', '）', '在', '水平', '射线', '上', '，', '适当', '分配', '折', '线', '的', '位置', '，', '确定', '直线', '的', '宽度', '和', '间隔', '。', '（', '3', '）', '在', '与', '水平', '射线', '垂直', '的', '深', '线', '上', '根据', '数据', '大小', '的', '具体', '情况', '，', '确定', '单位', '长度', '表示', '多少', '。', '（', '4', '）', '按照', '数据', '的', '大小', '描出', '各', '点', '，', '再', '用', '线段', '顺次', '连接', '起来', '，', '

# 图谱扩展